### Effective key transition analysis 1

- key position adjusted in increments of 0.25 mm
- vernier scale and Helmholtz resonance analysis used for calibration
- key manipulated by mechanical device with **flat area** finger
- infrared sensor (IR LED + Photoreceptive Transistor) calibrated for key positions

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import audacity
import TransferFunctions as tf
import SoundUtils as sut
import scipy.signal as sig

%matplotlib notebook

In [3]:
acquisitions = ["2201", "2301", "2301-2"]
runs = 17
dfiles = []

for acq in acquisitions:
    for r in range(runs):
        acqfiles = ['ALL_DATA/transfer/b-foot/%s_closing_%d.aup' % (acq, r)]
        dfiles.append(acqfiles)
        
reffile = "ALL_DATA/transfer/b-foot/2tieclip_reference_sines.aup"

In [4]:
def effective_transition(aupfile, freqIR, nfft, ch_noise=0, ch_IR=1, ch_ext=2, ch_int=3):
    '''
    Main routine for Effective Key Transition analysis.
    Uses transfer function analysis to determine when
    effective key state changes from open to closed.
    
    Input
    -----
    aupfile : audacity project file
    ch_noise, ch_IR, ch_int, ch_ext : (optional) channel numbers
        for noise, infrared, internal mic and external mic tracks
    IRfreq : frequency of infrared signal modulation
    nfft : window length for transfer function calculation
    
    Output
    ------
    returns a dictionary of the data with the following keys:
    'tf' = transfer function (internal/external response)
    'coh' = coherence (internal vs external)
    'int_mic' = internal response
    'ext_mic' = external responce
    '''
    auf = audacity.Aup(aupfile)
    print(aupfile)
    sr = auf.rate
    rawdata = []
    maxlen = 0
    for chno in range(auf.nchannels):
        rawdata.append(auf.get_channel_data(chno))
        maxlen = max(maxlen, len(rawdata[-1]))

    data = np.zeros((maxlen, len(rawdata)))
    for chno, chdata in enumerate(rawdata):
        data[:len(chdata), chno] = chdata
        
    src = data[:,ch_noise]
    int_ = data[:,ch_int]
    ext_ = data[:,ch_ext]
    IR = data[:,ch_IR]

    delay = tf.determineDelay(
        src/np.mean(src),int_/np.mean(int_),maxdel=2**15)
    print("Delay: %d samples"%delay)
    src = np.roll(src, delay)
    
    tfxy,ff = tf.tfe(int_,ext_,Fs=sr,NFFT=nfft)
    ff,coh = sig.coherence(int_,ext_,fs=sr,nperseg=nfft)
    datadict = {'tf':tfxy,'coh':coh}
    
    for chname, chdata in zip(["int_", "ext_"], [int_, ext_]):
        tfxy,ff = tf.tfe(chdata,src,Fs=sr,NFFT=nfft)
        datadict['%smic'%(chname)] = tfxy
        
    datadict["ir_RMS"] = np.sqrt(np.mean((IR-np.mean(IR))**2))
    return datadict

In [5]:
freqIR = 9820
nfft = 1024*2

refresult = effective_transition(reffile, freqIR, nfft)

ALL_DATA/transfer/b-foot/2tieclip_reference_sines.aup
Delay: 1763 samples


In [6]:
results = np.empty((len(acquisitions), runs))
for acqfiles in dfiles:
    for file in acqfiles:
        filedata = effective_transition(file, freqIR, nfft)
        results = np.append(results, [filedata])

ALL_DATA/transfer/b-foot/2201_closing_0.aup
Delay: 1828 samples
ALL_DATA/transfer/b-foot/2201_closing_1.aup
Delay: 1828 samples
ALL_DATA/transfer/b-foot/2201_closing_2.aup
Delay: 7561 samples
ALL_DATA/transfer/b-foot/2201_closing_3.aup
Delay: 7561 samples
ALL_DATA/transfer/b-foot/2201_closing_4.aup
Delay: 7561 samples
ALL_DATA/transfer/b-foot/2201_closing_5.aup
Delay: 7562 samples
ALL_DATA/transfer/b-foot/2201_closing_6.aup
Delay: 7562 samples
ALL_DATA/transfer/b-foot/2201_closing_7.aup
Delay: 7562 samples
ALL_DATA/transfer/b-foot/2201_closing_8.aup
Delay: 7536 samples
ALL_DATA/transfer/b-foot/2201_closing_9.aup
Delay: 7536 samples
ALL_DATA/transfer/b-foot/2201_closing_10.aup
Delay: 7537 samples
ALL_DATA/transfer/b-foot/2201_closing_11.aup
Delay: 1775 samples
ALL_DATA/transfer/b-foot/2201_closing_12.aup
Delay: 2077 samples
ALL_DATA/transfer/b-foot/2201_closing_13.aup
Delay: 2076 samples
ALL_DATA/transfer/b-foot/2201_closing_14.aup
Delay: 2076 samples
ALL_DATA/transfer/b-foot/2201_closi

In [21]:
print(results[0])

# plt.figure()
# for acqresults in results:
#     print(acqresults)
#     plt.plot(np.arange(17), [result['ir_RMS'] for result in acqresults], 'o-')

6.94446804248514e-310
